<h1>Creating Graphs</h1>

In [21]:
!pip install powerlaw

In [22]:
import json

with open("/kaggle/input/cic-ids-20117/df_properties.json", "r") as f:
    df_properties = json.load(f)

print("📘 Dataset Properties:")
for k, v in df_properties.items():
    print(f"{k}: {v}")


📘 Dataset Properties:
name: cic_ids_2017
length: 2286203
num_benign: 1737235
percentage_of_benign_records: 75.98778411190958
num_attack: 548968
percentage_of_attack_records: 24.012215888090427
attacks: ['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration', 'Web Attack � Brute Force', 'Web Attack � XSS', 'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator', 'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed']
weak_columns: ['Bwd Pkt Len Max', 'Fwd PSH Flags', 'Bwd Pkt Len Mean', 'RST Flag Cnt', 'TotLen Fwd Pkts', 'Fwd Pkt Len Max', 'Subflow Fwd Pkts', 'Pkt Len Mean', 'Fwd Blk Rate Avg', 'Bwd Pkts/b Avg', 'Flow IAT Mean', 'Bwd IAT Std', 'Idle Mean', 'Fwd URG Flags', 'Bwd Byts/b Avg', 'Fwd Pkt Len Std', 'Bwd Blk Rate Avg', 'Fwd Pkt Len Mean', 'Fwd Pkts/b Avg', 'Fwd IAT Mean', 'Subflow Bwd Byts', 'Active Std', 'Flow IAT Max', 'Idle Max', 'Bwd IAT Mean', 'Pkt Len Var', 'Subflow Bwd Pkts', 'TotLen Bwd Pkts', 'Pkt Size Avg', 'Fwd IAT Tot', 'Bwd PSH Flags', 'Bwd

In [23]:
import pickle

with open("/kaggle/input/cic-ids-20117/labels_names.pkl", "rb") as f:
    labels_names = pickle.load(f)
print(labels_names)
print("\n🏷️ Label Class Mapping:")
# for k, v in labels_names.items():
#     print(f"{k}: {v}")


[{0: 'BENIGN', 1: 'Bot', 2: 'DDoS', 3: 'DoS GoldenEye', 4: 'DoS Hulk', 5: 'DoS Slowhttptest', 6: 'DoS slowloris', 7: 'FTP-Patator', 8: 'Heartbleed', 9: 'Infiltration', 10: 'PortScan', 11: 'SSH-Patator', 12: 'Web Attack � Brute Force', 13: 'Web Attack � Sql Injection', 14: 'Web Attack � XSS'}, array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)]

🏷️ Label Class Mapping:


In [24]:
# !pip install igraph
# !pip install networkx 

In [25]:
import os
import pickle
import random
import socket
import struct

import sys
import pandas as pd
sys.path.append("/kaggle/input/gnn-nids/pytorch/default/1")

import networkx as nx
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from src.dataset.dataset_info import datasets
from src.graph.graph_construction import create_weightless_window_graph
from src.graph.graph_measures import calculate_graph_measures
from src.graph.centralities import add_centralities, add_centralities_as_node_features
# from local_variables import local_datasets_path
local_datasets_path = "/kaggle/input/"
original_datasets_files_path = "/kaggle/input/cic-ids-2017-gnn-2"
# /kaggle/input/cic-ids-2017-gnn-2


In [26]:
multi_class = True

use_node_features = True

use_port_in_address = False

generated_ips = False

graph_type = "flow"
# graph_type = "window"
# graph_type = "line"

# window_size= 400
window_size= 500
# print("1. flow sorted")
# print("2. flow using node features unsorted") 
print("3. flow using node features sorted")
# 4. flow using port numbers sorted
sort_timestamp = True

# k_fold = None
# k_fold = 5

validation_size = 0.1
test_size = 0.1

cn_measures = ["betweenness", "degree", "pagerank", "closeness", "k_truss"]
# cn_measures = ["betweenness", "degree", "closeness"]

network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank', 'src_closeness', 'dst_closeness', 'src_k_truss', 'dst_k_truss']
# network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank']

3. flow using node features sorted


In [27]:
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
# name = "cic_ids_2017_5_percent"
name = "cic_ids_2017"
# name = "cic_bot_iot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"

dataset = datasets[name]

In [28]:
g_type = ""
if graph_type == "flow":
    g_type = "flow"
elif graph_type == "line":
    g_type = f"line_graph_{window_size}"
elif graph_type == "window":
    g_type = f"window_graph_{window_size}"
    
if multi_class:
    g_type += "__multi_class"
    
if use_node_features:
    g_type += "__n_feats"
    
# if k_fold:
#     g_type += f"__{k_fold}_fold"
    
if use_port_in_address:
    g_type += "__ports"
    
if generated_ips:
    g_type += "__generated_ips"
    
if sort_timestamp:
    g_type += "__sorted"
else:
    g_type += "__unsorted"
# ****************** ALTERED    



dataset_path = os.path.join(local_datasets_path,name)
folder_path = os.path.join(dataset_path, g_type)
if name == "cic_ids_2017":
    name_ds = "cic-ids-2017-gnn-2"
    dataset_path = os.path.join(local_datasets_path,name_ds)
    # folder_path = os.path.join(local_datasets_path,name, g_type)
    folder_path= os.path.join("/kaggle/working/",name,g_type)

print(folder_path)
print(dataset_path)

/kaggle/working/cic_ids_2017/flow__multi_class__unsorted
/kaggle/input/cic-ids-2017-gnn-2


In [29]:
# if name== "cic_ids_2017":
#     df=pd.read_parquet(os.path.join(dataset_path, f"{name}.parquet"))
# else:
df = pd.read_parquet(os.path.join(dataset_path, f"{name}.parquet"))

In [30]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
0,758097,104.16.207.165,443.0,192.168.10.5,54865.0,6.0,2017-07-07 03:30:00+00:00,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
1,758390,104.16.28.216,80.0,192.168.10.5,55054.0,6.0,2017-07-07 03:30:00+00:00,109.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
2,758391,104.16.28.216,80.0,192.168.10.5,55055.0,6.0,2017-07-07 03:30:00+00:00,52.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
3,422766,104.17.241.25,443.0,192.168.10.16,46236.0,6.0,2017-07-07 03:30:00+00:00,34.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
4,758524,104.19.196.102,443.0,192.168.10.5,54863.0,6.0,2017-07-07 03:30:00+00:00,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0


In [31]:
cols_to_norm = list(set(list(df.columns))  - set(list([dataset.label_col, dataset.class_num_col])) - set(dataset.drop_columns)  - set(dataset.weak_columns))

In [32]:
df[dataset.label_col].value_counts()

Label
0    1737235
1     548968
Name: count, dtype: int64

In [33]:
if generated_ips:
    df[dataset.src_ip_col] = df[dataset.src_ip_col].apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

In [34]:
if sort_timestamp:
    # df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)
    df.sort_values(dataset.timestamp_col, inplace=True)

In [35]:
if use_port_in_address:
    df[dataset.src_port_col] = df[dataset.src_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.src_ip_col] = df[dataset.src_ip_col] + ':' + df[dataset.src_port_col]

    df[dataset.dst_port_col] = df[dataset.dst_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.dst_ip_col] = df[dataset.dst_ip_col] + ':' + df[dataset.dst_port_col]

In [36]:
df.head(20)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
0,758097,104.16.207.165,443.0,192.168.10.5,54865.0,6.0,2017-07-07 03:30:00+00:00,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
1,758390,104.16.28.216,80.0,192.168.10.5,55054.0,6.0,2017-07-07 03:30:00+00:00,109.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
2,758391,104.16.28.216,80.0,192.168.10.5,55055.0,6.0,2017-07-07 03:30:00+00:00,52.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
3,422766,104.17.241.25,443.0,192.168.10.16,46236.0,6.0,2017-07-07 03:30:00+00:00,34.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
4,758524,104.19.196.102,443.0,192.168.10.5,54863.0,6.0,2017-07-07 03:30:00+00:00,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
5,758666,104.20.10.120,443.0,192.168.10.5,54871.0,6.0,2017-07-07 03:30:00+00:00,1022.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
6,758672,104.20.10.120,443.0,192.168.10.5,54925.0,6.0,2017-07-07 03:30:00+00:00,4.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
7,758672,104.20.10.120,443.0,192.168.10.5,54925.0,6.0,2017-07-07 03:30:00+00:00,42.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
8,809266,104.28.13.116,443.0,192.168.10.8,9282.0,6.0,2017-07-07 03:30:00+00:00,4.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0
9,759808,104.97.123.193,443.0,192.168.10.5,55153.0,6.0,2017-07-07 03:30:00+00:00,4.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,BENIGN,0


In [37]:
print(dataset.class_num_col)

Class


In [38]:
if multi_class:
    y = df[dataset.class_num_col]
else:
    y = df[dataset.label_col]

if sort_timestamp:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size)
else:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size, random_state=13, stratify=y)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size, random_state=13, stratify=y_tr)

del df

In [39]:
if graph_type == "line" and use_node_features:
    add_centralities(df = X_train, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_val, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_test, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    cols_to_norm = list(set(cols_to_norm) | set(network_features))
    

In [40]:
scaler = StandardScaler()

X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
X_train['h'] = X_train[ cols_to_norm ].values.tolist()

cols_to_drop = list(set(list(X_train.columns)) - set(list([dataset.label_col, dataset.src_ip_col, dataset.dst_ip_col, dataset.class_num_col, 'h'])))
X_train.drop(cols_to_drop, axis=1, inplace=True)

X_val[cols_to_norm] = scaler.transform(X_val[cols_to_norm])
X_val['h'] = X_val[ cols_to_norm ].values.tolist()
X_val.drop(cols_to_drop, axis=1, inplace=True)

X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test[ cols_to_norm ].values.tolist()
X_test.drop(cols_to_drop, axis=1, inplace=True)

In [41]:
if graph_type == "window" or graph_type == "line":

    create_weightless_window_graph(
        df=X_train,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "training"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_val,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "validation"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_test,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "testing"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")

In [42]:
if graph_type == "flow":
	os.makedirs(folder_path, exist_ok=True)
	print(f"==>> X_train.shape: {X_train.shape}")
	print(f"==>> X_val.shape: {X_val.shape}")
	print(f"==>> X_test.shape: {X_test.shape}")


==>> X_train.shape: (1851823, 5)
==>> X_val.shape: (205759, 5)
==>> X_test.shape: (228621, 5)


In [43]:
if graph_type == "flow":
    graph_name = "training_graph"

    G = nx.from_pandas_edgelist(X_train, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

==>> calculated degrees, in 0.03391512300004251 seconds
==>> graph_measures: {'number_of_nodes': 16870, 'number_of_edges': 1851823, 'max_degree': 619539, 'avg_degree': 219.54036751630113, 'density': 0.006507213454155585}


In [44]:
if graph_type == "flow":
    graph_name = "validation_graph"

    G = nx.from_pandas_edgelist(X_val, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

==>> calculated degrees, in 0.013613691000045947 seconds
==>> graph_measures: {'number_of_nodes': 10955, 'number_of_edges': 205759, 'max_degree': 68543, 'avg_degree': 37.56439981743496, 'density': 0.0017146430444328537}


In [45]:
if graph_type == "flow":
    graph_name = "testing_graph"
    
    G = nx.from_pandas_edgelist(X_test, dataset.src_ip_col, dataset.dst_ip_col, ['h', dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    # graph_measures = calculate_graph_measures(nx.DiGraph(G_test), "datasets/" + name + "/testing_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")
    
    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

==>> calculated degrees, in 0.014158562000147867 seconds
==>> graph_measures: {'number_of_nodes': 11395, 'number_of_edges': 228621, 'max_degree': 76019, 'avg_degree': 40.12654673102238, 'density': 0.0017608630301484281}


In [47]:
# Save all installed package versions
!pip freeze > requirements_1_GNN.txt

# Display first few lines
!head -n 20 requirements.txt


head: cannot open 'requirements.txt' for reading: No such file or directory


In [48]:
import json

print("-------------------------- SAVING VARIABLES -------------------------- ")
data = {
    "dataset_path": dataset_path,
    "folder_path": folder_path,
    "name": name,
    "g_type": g_type
}

# Save to JSON
with open("config.json", "w") as f:
    json.dump(data, f, indent=4)


-------------------------- SAVING VARIABLES -------------------------- 


In [49]:
import psutil
import gc
ram_gb = psutil.virtual_memory().used / 1e9
print(f"Current RAM usage: {ram_gb:.2f} GB")
gc.collect()
print("-------------------------- CLEARING THE MEMORY -------------------------- ")
print("Clear all variables (like restarting kernel)")
%reset -f


Current RAM usage: 14.13 GB
-------------------------- CLEARING THE MEMORY -------------------------- 
Clear all variables (like restarting kernel)


In [50]:
import psutil
ram_gb = psutil.virtual_memory().used / 1e9
print(f"Current RAM usage: {ram_gb:.2f} GB")

Current RAM usage: 4.78 GB


<h1>TRAINNING</h1>

In [51]:
import json
print("-------------------------start Training-----------------------------")
print("-------------------------- RETRIVING VARIABLES --------------------------")
with open("config.json", "r") as f:
    config = json.load(f)

# Access the values
dataset_path = config["dataset_path"]
folder_path = config["folder_path"]
name = config["name"]
g_type = config["g_type"]


-------------------------start Training-----------------------------
-------------------------- RETRIVING VARIABLES --------------------------


In [52]:
print(folder_path)

/kaggle/working/cic_ids_2017/flow__multi_class__unsorted


In [53]:

# print(folder_path)
# print(g_type)
# dataset_name = "cic_ids_2017"
# # dataset_folder = "/kaggle/input/cic-ids-2017-gnn"
# dataset_folder = "/kaggle/input/cic-ids-2017-gnn-2"
# # grpah_folder="/kaggle/input/cic-ids-2017-gnnflow-multi-class-unsorted"
# graphs_folder=folder_path
# # /kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted
# # "/kaggle/input/cic-ids-2017-gnn"
# # /kaggle/input/d/mohammadfleity/cic-ids-2017-gnn/cic_ids_2017.parquet
# model_path = "/kaggle/input/gnn-nids-wandb/pytorch/default/6/"
dataset_name = name
dataset_folder = dataset_path
graph_folder=folder_path
model_path = "/kaggle/input/gnn-nids-wandb/pytorch/default/6/"
using_wandb = False
save_top_k = 1


# save_top_k = 4 save best 4 test model from the traininng based on f1_score 
# early_stopping_patience = max_epochs = 20
early_stopping_patience = max_epochs = 500
# early_stopping_patience = 200 all - epochs = 500 - lr = 0.005
learning_rate = 0.005
weight_decay = 0.01
ndim_out = [32, 32]
num_layers = 2
number_neighbors = [25, 10]
dropout = 0.5
residual = True
multi_class = True
aggregation = "mean"
# aggregation = "sum"

use_centralities_nfeats = False

# g_type = "flow"

# if multi_class:
#     g_type += "__multi_class"

# if use_centralities_nfeats:
#     g_type += "__n_feats"
    
# if sort_timestamp:
#     g_type += "__sorted"
# else:
#     g_type += "__unsorted"

In [54]:
pip uninstall -y torch torchvision torchaudio torchdata pytorch-lightning --quiet

Note: you may need to restart the kernel to use updated packages.


In [55]:
print("dgl")


dgl


In [56]:
# pip install dgl 

In [57]:
pip install dgl -f https://data.dgl.ai/wheels/torch-2.1/cu118/repo.html 

Looking in links: https://data.dgl.ai/wheels/torch-2.1/cu118/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.4/528.4 MB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 21.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 14.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [58]:
print("done_dgl")

done_dgl


In [59]:
pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 438.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 103.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 6.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 20.6 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [60]:
import torch
print("torch version:", torch.__version__)
print("CUDA available?", torch.cuda.is_available())

torch version: 2.1.0+cu118
CUDA available? True


In [61]:
!apt update -qq 2>/dev/null >/dev/null;
!apt install -y libcusparse11 -qq 2>/dev/null >/dev/null;


In [62]:
pip install --upgrade -q pytorch-lightning==2.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.4/819.4 kB 15.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [63]:
!pip install --upgrade -q wandb==0.19.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 68.6 MB/s eta 0:00:00:00:0100:01


In [64]:
# import wandb
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("mohammad_wandb_secret")

wandb.login(key=secret_value_0)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammad-fleity (mohammad-fleity-lebanese-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [65]:
import os
import json
import pickle
import time
import timeit
import importlib.util
import sys
import random

import numpy as np

os.environ["DGLBACKEND"] = "pytorch"

import torch.nn as nn
import torch
import warnings
import wandb
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

seed = 42  # or any constant value
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

run_dtime = time.strftime("%Y%m%d-%H%M%S")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cic-ids-2017-gnn-2/df_properties.json
/kaggle/input/cic-ids-2017-gnn-2/cic_ids_2017.parquet
/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
/kaggle/input/githubrepofiles/pytorch/default/1/time_efficiency.ipynb
/kaggle/input/githubrepofiles/pytorch/default/1/.gitignore
/kaggle/input/githubrepofiles/pytorch/default/1/prepare_graph_files.ipynb
/kaggle/input/githubrepofiles/pytorch/default/1/main.py
/kaggle/input/githubrepofiles/pytorch/default/1/README.md
/kaggle/input/githubrepofiles/pytorch/default/1/dataset_properties.ipynb
/kaggle/input/githubrepofiles/pytorch/default/1/pre_processing.ipynb
/kaggle/input/githubrepofiles/pytorch/default/1/concat_properties.py
/kaggle/input/githubrepofiles/pytorch/default/1/requirements.txt
/kaggle/input/githubrepofiles/pytorch/default/1/graph_properties.ipynb
/kaggle/input/githubrepofiles/pytorch/default/1/results_analysis.ipynb
/kaggle/input/githubrepofiles/pytorch/default/1/testing_dfs/cic_ton_iot_5_percent.parquet
/kaggle/input/githu

In [66]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/config.json
/kaggle/working/requirements_1_GNN.txt
/kaggle/working/cic_ids_2017/flow__multi_class__unsorted/validation_graph_measures.json
/kaggle/working/cic_ids_2017/flow__multi_class__unsorted/testing_graph_measures.json
/kaggle/working/cic_ids_2017/flow__multi_class__unsorted/validation_graph.pkl
/kaggle/working/cic_ids_2017/flow__multi_class__unsorted/training_graph_measures.json
/kaggle/working/cic_ids_2017/flow__multi_class__unsorted/testing_graph.pkl
/kaggle/working/cic_ids_2017/flow__multi_class__unsorted/training_graph.pkl


In [67]:
# pip install torchdata

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [69]:
# import kagglehub

# # Download latest version
# path = kagglehub.model_download("mortadaphdtermos/gnn-nids-wandb/pyTorch/default")

# print("Path to model files:", path)

In [70]:
# pip install torchdata==0.7.0

In [71]:
# pip install torch.utils

In [72]:
def add_lib(module_name, path):
    spec = importlib.util.spec_from_file_location(module_name, path)
    dataset_info = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = dataset_info
    spec.loader.exec_module(dataset_info)

add_lib("dataset_info", model_path + "dataset_info.py")
add_lib("lightning_data", model_path + "lightning_data.py")
add_lib("lightning_model", model_path + "lightning_model.py")
add_lib("models", model_path + "models.py")

In [73]:
from models import EGAT, EGCN, EGRAPHSAGE
from lightning_model import GraphModel
from lightning_data import GraphDataModule
from dataset_info import datasets
import time
import psutil
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [74]:
# pip install torchmetrics

In [75]:
# pip install torch_geometric

<h2>E_GAT</h2>

In [76]:
# pip install _XLAC

In [77]:
# import torch_xla.core.xla_model as xm # FOR TPU ONLY

# # For XLA tensors
# xla_tensor = self.graph.edata[self.class_num_col]
# target = xm.to_cpu(xla_tensor).detach().numpy()


In [78]:
import os
import pickle
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import numpy as np
import wandb
import csv

In [79]:
# Save all installed package versions
!pip freeze > requirements_2_GNN.txt

# Display first few lines
!head -n 20 requirements.txt


head: cannot open 'requirements.txt' for reading: No such file or directory


In [ ]:
warnings.filterwarnings("ignore", ".*does not have many workers.*")

dataset = datasets[dataset_name]

# Hyperparameters
aggregation = "mean"
activation = F.relu

graphs_folder = folder_path
# if dataset_name == "cic_ids_2017":
    # graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__sorted"

logs_folder = os.path.join("logs", dataset.name)
os.makedirs(logs_folder, exist_ok=True)

wandb_runs_path = os.path.join("logs", "wandb_runs")
os.makedirs(wandb_runs_path, exist_ok=True)

labels_mapping = {0: "Normal", 1: "Attack"}
num_classes = 2
if multi_class:
    with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
        labels_names = pickle.load(f)
    labels_mapping = labels_names[0]
    num_classes = len(labels_mapping)

dataset_kwargs = dict(
    use_node_features=use_centralities_nfeats,
    multi_class=True,
    using_masking=False,
    masked_class=2,
    num_workers=0,
    label_col=dataset.label_col,
    class_num_col=dataset.class_num_col,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

data_module = GraphDataModule(graphs_folder, batch_size=1, **dataset_kwargs)
data_module.setup()

ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
edim = next(iter(data_module.train_dataloader())).edata["h"].shape[-1]

my_models = {
    "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes),
    f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,residual, num_classes, num_neighbors=None, aggregation=aggregation),
    f"e_graphsage_{aggregation}": EGRAPHSAGE(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=number_neighbors,
        aggregation=aggregation
    ),
    f"e_graphsage_sum_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,residual, num_classes, num_neighbors=None, aggregation="sum"),
    f"e_graphsage_sum": EGRAPHSAGE(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=number_neighbors,
        aggregation="sum"
    ),
    "e_gat_sampling": EGAT(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=number_neighbors
    ),
    "e_gat_no_sampling": EGAT(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=None
    ),
}

criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

for model_name, model in my_models.items():

    config = {
        "run_dtime": run_dtime,
        "type": "GNN",
        "model_name": model_name,
        "max_epochs": max_epochs,
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "ndim_out": ndim_out,
        "num_layers": num_layers,
        "number_neighbors": number_neighbors,
        "activation": activation.__name__,
        "dropout": dropout,
        "residual": residual,
        "multi_class": multi_class,
        "aggregation": aggregation,
        "early_stopping_patience": early_stopping_patience,
        "use_centralities_nfeats": use_centralities_nfeats,
    }

    graph_model = GraphModel(
        model, criterion, learning_rate, config, model_name,
        labels_mapping, weight_decay=weight_decay,
        using_wandb=using_wandb, norm=False, multi_class=True
    )

    if using_wandb:
        wandb_logger = WandbLogger(
            project=f"GNN-Analysis-{dataset.name}",  # Project Name Wandb
            name=f"{model_name}---{g_type}",
            config=config,
            save_dir=wandb_runs_path,
        )
    else:
        wandb_logger = None

    f1_checkpoint_callback = ModelCheckpoint(
        monitor="val_f1_score",
        mode="max",
        filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
        save_top_k=save_top_k,
        verbose=False,
    )

    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        mode="min",
        patience=early_stopping_patience,
        verbose=False,
    )

    process = psutil.Process(os.getpid())  # current process
    start_time = time.time()
    start_mem = process.memory_info().rss / (1024 ** 2)  # in MB
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        num_sanity_val_steps=0,
        log_every_n_steps=0,
        callbacks=[f1_checkpoint_callback, early_stopping_callback],
        default_root_dir=logs_folder,
        logger=wandb_logger if using_wandb else None,
    )

    trainer.fit(graph_model, datamodule=data_module)
    
    test_results = []
    print(process)
    for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
        graph_model.test_prefix = f"best_f1_{i}"
        # Start time & memory
        results = trainer.test(graph_model, datamodule=data_module, ckpt_path=k)
        test_results.append(results[0][f"best_f1_{i}_test_f1"])

        # End time & memory
        print(f"\n--- Test Run {i+1} ---")
        print("graph_model: ", graph_model)
        print("data_module: ", data_module)
        print("results: ", results)
        logs = {
        "median_f1_of_best_f1": np.median(test_results),
        "max_f1_of_best_f1": np.max(test_results),
        "avg_f1_of_best_f1": np.mean(test_results),
            
    }
    print("test_results: ", test_results)
    end_time = time.time()
    end_mem = process.memory_info().rss / (1024 ** 2)  # in MB
    
    elapsed_time = end_time - start_time
    mem_used = end_mem - start_mem
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    print(f"Memory used: {mem_used:.2f} MB (current RSS: {end_mem:.2f} MB)")
    # metric_dir= "/kaggle/working/temp"
    # os.makedirs(metric_dir, exist_ok=True)
    
    # csv_path = os.path.join(metric_dir, f"{model_name}_metrics.csv")
    
    # with open(csv_path, mode="w", newline="") as f:
    #     writer = csv.writer(f)
    #     writer.writerow(["TEST_ACCURACY", test_results[0]])
    #     writer.writerow(["MEMORY_USED", f"{mem_used:.2f} MB"])
    #     writer.writerow(["TIME_CONSUMED", f"{elapsed_time:.2f} s."])
    
    # print(f"Metrics saved to {csv_path}")
    metric_dir = "/kaggle/working/temp"
    os.makedirs(metric_dir, exist_ok=True)
    
    json_path = os.path.join(metric_dir, f"{model_name}_metrics.json")
    
    # Create dataframe with a single row
    df = pd.DataFrame([{
        # "model_name": model_name,
        "TEST_ACCURACY": test_results[0],
        "MEMORY_USED": f"{mem_used:.2f} MB",
        "TIME_CONSUMED": f"{elapsed_time:.2f} s."
    }])
    
    # Save to JSON (records = list of dicts)
    df.to_json(json_path, orient="records", indent=4)
    
    print(f"Metrics saved to {json_path}")


    if using_wandb:
        wandb.log(logs)
        wandb.finish()
    else:
        trainer.logger.log_metrics(logs, step=trainer.global_step)


In [ ]:
import os
import json
import pandas as pd

# Your label mapping (from the print output you shared)
labels_map = {
    "0": "BENIGN",
    "1": "Bot",
    "2": "DDoS",
    "3": "DoS GoldenEye",
    "4": "DoS Hulk",
    "5": "DoS Slowhttptest",
    "6": "DoS slowloris",
    "7": "FTP-Patator",
    "8": "Heartbleed",
    "9": "Infiltration",
    "10": "PortScan",
    "11": "SSH-Patator",
    "12": "Web Attack \ufffd Brute Force",
    "13": "Web Attack \ufffd Sql Injection",
    "14": "Web Attack \ufffd XSS"
}
def json_to_csv_per_file(json_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(json_folder):
        if filename.endswith("_results.json"):
            no_ext = filename.replace("_results.json", "")
            # Remove the trailing "_metrics"
            print(no_ext)
            model_name=no_ext
            metric_path = os.path.join(json_folder,f"{no_ext}_metrics.json")
            print(metric_path)
            filepath = os.path.join(json_folder, filename)
            with open(filepath, "r") as f:
                data = json.load(f)
            rows = []
            # --- Per-class metrics ---
            classification_report=data["classification_report"]
            # print(classification_report)
            if "results_fpr_fnr" in data and "per_class" in data["results_fpr_fnr"]:
                # print(data["results_fpr_fnr"])
                for cls, metrics in data["results_fpr_fnr"]["per_class"].items():
                    attack=labels_map.get(cls,cls)
                    print(f"{attack}")
                    metrics_d=classification_report.get(attack)
                    row = {
                        "ATTACK TYPE": attack,
                        "precision": metrics_d.get("precision"),
                        "recall": metrics_d.get("recall"),
                        "f1_score": metrics_d.get("f1-score"),
                        "support": metrics_d.get("support"),
                        "FPR": metrics.get("FPR"),
                        "FNR": metrics.get("FNR"),
                    }
                    # print()
                    # print(metrics_d.get("precision"))
                    # print()
                    rows.append(row)

            df_classes = pd.DataFrame(rows)
            weighted_avg=classification_report['weighted avg']
            # print(data)
            # print(data['classification_report']['accuracy'])
            # --- Global / test metrics ---
            with open(metric_path, "r") as f:
                device_data = json.load(f)
            print(device_data)
            MEMORY_USED=device_data[0]["MEMORY_USED"]
            TIME_CONSUMED=device_data[0]["TIME_CONSUMED"]
            # print("Memory:", MEMORY_USED)
            # print("Time:", TIME_CONSUMED)
            global_metrics = {
                'model_name':model_name,
                "test_weighted_f1": data.get("test_weighted_f1"),
                'test_accuracy':data['classification_report']['accuracy'],
                'precision': weighted_avg['precision'],
                'recall': weighted_avg['recall'], 
                'support': weighted_avg['support'],
                # "test_fpr": data.get("test_fpr"),
                # "test_fnr": data.get("test_fnr"),
                "global_FPR": data["results_fpr_fnr"]["global"].get("FPR") if "results_fpr_fnr" in data and "global" in data["results_fpr_fnr"] else None,
                "global_FNR": data["results_fpr_fnr"]["global"].get("FNR") if "results_fpr_fnr" in data and "global" in data["results_fpr_fnr"] else None,
                "time_consumption":TIME_CONSUMED,
                "memory_consumption":MEMORY_USED,
            }
            
            print(global_metrics)
            df_global = pd.DataFrame([global_metrics])
            # --- Save CSV ---
            csv_output = os.path.join(output_folder, filename.replace(".json", ".csv"))
            with open(csv_output, "w") as f:
                f.write("### Per-class Metrics\n")
            df_classes.to_csv(csv_output, mode="a", index=False)
            with open(csv_output, "a") as f:
                f.write("\n### Global Metrics\n")
            df_global.to_csv(csv_output, mode="a", index=False)

            print(f"Saved {csv_output}")
json_folder = "/kaggle/working/temp/"
output_folder = "/kaggle/working/temp/csv_results"
json_to_csv_per_file(json_folder, output_folder)



<h2>E_GAT Sampling</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
# #     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE mean</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
# #     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                                          residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #               residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE sum</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="sum"

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
#     # graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                                          residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #               residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE Sampling mean</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="mean"

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
#     # graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                              residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE Sampling sum</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="sum"

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
#     # graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                              residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>E_GCN</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="mean"

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
# #     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#                   dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)